<a href="https://colab.research.google.com/github/MPdeM/Final_Project/blob/master/machinelearning/nlp_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk

### **Dependencies**

In [25]:
# Dependencies
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jobrien1726/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Load/Clean Data**

In [26]:
# Read in csv file
url = "https://job-postings-dataviz.s3.amazonaws.com/fake_jobs_clean.csv"

fake_jobs_df = pd.read_csv(url, sep=",",  encoding = "UTF-8")
fake_jobs_df

,job_id,city,state/province,country,title,department,industry,function,salary_range,salary_provided,...,requirements,benefits,benefits_provided,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,1,New York,NY,US,Marketing Intern,Marketing,NaN,Marketing,NaN,0,...,Experience with content management systems a m...,NaN,0,0,1,0,Other,Internship,NaN,0
1,2,Auckland,,NZ,Customer Service - Cloud Video Production,Success,Marketing and Advertising,Customer Service,NaN,0,...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,1,0,1,0,Full-time,Not Applicable,NaN,0
2,3,Wever,IA,US,Commissioning Machinery Assistant (CMA),NaN,NaN,NaN,NaN,0,...,Implement pre-commissioning and commissioning ...,NaN,0,0,1,0,NaN,NaN,NaN,0
3,4,Washington,DC,US,Account Executive - Washington DC,Sales,Computer Software,Sales,NaN,0,...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,1,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0
4,5,Fort Worth,FL,US,Bill Review Manager,NaN,Hospital & Health Care,Health Care Provider,NaN,0,...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Toronto,ON,CA,Account Director - Distribution,Sales,Computer Software,Sales,NaN,0,...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,1,0,1,1,Full-time,Mid-Senior level,NaN,0
17876,17877,Philadelphia,PA,US,Payroll Accountant,Accounting,Internet,Accounting/Auditing,NaN,0,...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0
17877,17878,Houston,TX,US,Project Cost Control Staff Engineer - Cost Con...,NaN,NaN,NaN,NaN,0,...,At least 12 years professional experience.Abil...,NaN,0,0,0,0,Full-time,NaN,NaN,0
17878,17879,Lagos,LA,NG,Graphic Designer,NaN,Graphic Design,Design,NaN,0,...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,1,0,0,1,Contract,Not Applicable,Professional,0


In [27]:
# Take a look at job description and fraudulent cols
df = fake_jobs_df[['description', 'fraudulent']]
df

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0
...,...,...
17875,Just in case this is the first time you’ve vis...,0
17876,The Payroll Accountant will focus primarily on...,0
17877,Experienced Project Cost Control Staff Enginee...,0
17878,Nemsia Studios is looking for an experienced v...,0


In [28]:
# Count how many Real vs Fraudulent postings 
from collections import Counter
print(Counter(df['fraudulent'].values))

Counter({0: 17014, 1: 866})


In [29]:
# Drop duplicate descriptions
df = df.drop_duplicates()
df

,description,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",0
1,Organised - Focused - Vibrant - Awesome!Do you...,0
2,"Our client, located in Houston, is actively se...",0
3,THE COMPANY: ESRI – Environmental Systems Rese...,0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,0
...,...,...
17873,RESPONSIBILITIES:Will facilitate the recruitin...,0
17875,Just in case this is the first time you’ve vis...,0
17876,The Payroll Accountant will focus primarily on...,0
17877,Experienced Project Cost Control Staff Enginee...,0


In [30]:
# Count how many Real vs Fraudulent postings 
print(Counter(df['fraudulent'].values))

Counter({0: 14171, 1: 632})


In [31]:
# Make sure number of real vs fraudulent are more balanced
# Using UNDERsampling for balancing of classification groups
df_fraud= df[df['fraudulent'] == 1] 
df_normal = df[df['fraudulent'] == 0] 

df_normal = df_normal.sample(n=(9*len(df_fraud)), random_state=22)
df = df_normal.append(df_fraud)

df

,description,fraudulent
13898,Currently hiring per-diem caregivers to work o...,0
1688,"Μεγαλώνουμε την ομάδα μας,και χρειαζόμαστε ένα...",0
13100,PowToon is looking for a team-oriented Marketi...,0
5579,Wealth Management Advisor We are continuing ou...,0
15961,RECRUITING MILITARY VETERANSLooking to offer o...,0
...,...,...
17827,Student Positions Part-Time and Full-Time.You ...,1
17828,LEARN TO EARN AN EXECUTIVE LEVEL INCOMEFULL TR...,1
17829,inFullMobile Sp. z o.o. is a mobile software d...,1
17830,JOB DESCRIPTIONWe are seeking a full time payr...,1


In [32]:
# Shuffle the dataframe so fraudulent postings more evenly distributed 
from sklearn.utils import shuffle
df = shuffle(df, random_state=22)
df = df.reset_index(drop=True)
df

,description,fraudulent
0,The Customer Service Associate will be based ...,0
1,Solid seeks to recruit a Digital Account Super...,0
2,(We have more than 1500+ Job openings in our w...,0
3,Support the Pricing Intelligence initiative by...,0
4,Position Summary: Provide integrity engineeri...,1
...,...,...
6315,Role: Receptionist/Administrator Location: Cam...,0
6316,CTO for a tourism start-upWe are just now star...,0
6317,Senior FrontEnd Developer for our small and ta...,0
6318,We have an immediate opening for a receptionis...,1


In [33]:
# Explicitly define data types
df['description'] = df['description'].astype(str)
df['fraudulent'] = df['fraudulent'].astype(int)

In [34]:
# Datatypes
df.dtypes

description    object
fraudulent      int64
dtype: object

In [35]:
# Count Real vs Fraudulent again to make sure we've achieved the desired result
print(Counter(df['fraudulent'].values))

Counter({0: 5688, 1: 632})


## **Data Pre-Processing**

In [36]:
# Change all the text to lowercase
df['description'] = [x.lower() for x in df['description']]
df['description']

0        the customer service associate will be based ...
1       solid seeks to recruit a digital account super...
2       (we have more than 1500+ job openings in our w...
3       support the pricing intelligence initiative by...
4       position summary:  provide integrity engineeri...
                              ...                        
6315    role: receptionist/administrator location: cam...
6316    cto for a tourism start-upwe are just now star...
6317    senior frontend developer for our small and ta...
6318    we have an immediate opening for a receptionis...
6319    qubit: cutting edge big data engineeringat qub...
Name: description, Length: 6320, dtype: object

In [37]:
# Tokenization
df['description']= [word_tokenize(x) for x in df['description']]

In [38]:
# Remove Stop words and Non-Numeric. Perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index,entry in enumerate(df['description']):
    # Declare an Empty List to store the words that follow the rules for this step
    Final_words = []

    # Initialize WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()

    # pos_tag function to provide the 'tag' i.e if the word is Noun(N), Verb(V), ADJ(J) or ADV(R) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)

    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'text_final'] = str(Final_words)

df

,description,fraudulent,text_final
0,"[the, customer, service, associate, will, be, ...",0,"['customer', 'service', 'associate', 'base', '..."
1,"[solid, seeks, to, recruit, a, digital, accoun...",0,"['solid', 'seek', 'recruit', 'digital', 'accou..."
2,"[(, we, have, more, than, 1500+, job, openings...",0,"['job', 'opening', 'website', 'relevant', 'job..."
3,"[support, the, pricing, intelligence, initiati...",0,"['support', 'pricing', 'intelligence', 'initia..."
4,"[position, summary, :, provide, integrity, eng...",1,"['position', 'summary', 'provide', 'integrity'..."
...,...,...,...
6315,"[role, :, receptionist/administrator, location...",0,"['role', 'location', 'camden', 'londoncompany'..."
6316,"[cto, for, a, tourism, start-upwe, are, just, ...",0,"['cto', 'tourism', 'start', 'want', 'join', 'a..."
6317,"[senior, frontend, developer, for, our, small,...",0,"['senior', 'frontend', 'developer', 'small', '..."
6318,"[we, have, an, immediate, opening, for, a, rec...",1,"['immediate', 'opening', 'year', 'experience',..."


In [39]:
# Reorganize Columns
df = df[['description','text_final','fraudulent']]
df

,description,text_final,fraudulent
0,"[the, customer, service, associate, will, be, ...","['customer', 'service', 'associate', 'base', '...",0
1,"[solid, seeks, to, recruit, a, digital, accoun...","['solid', 'seek', 'recruit', 'digital', 'accou...",0
2,"[(, we, have, more, than, 1500+, job, openings...","['job', 'opening', 'website', 'relevant', 'job...",0
3,"[support, the, pricing, intelligence, initiati...","['support', 'pricing', 'intelligence', 'initia...",0
4,"[position, summary, :, provide, integrity, eng...","['position', 'summary', 'provide', 'integrity'...",1
...,...,...,...
6315,"[role, :, receptionist/administrator, location...","['role', 'location', 'camden', 'londoncompany'...",0
6316,"[cto, for, a, tourism, start-upwe, are, just, ...","['cto', 'tourism', 'start', 'want', 'join', 'a...",0
6317,"[senior, frontend, developer, for, our, small,...","['senior', 'frontend', 'developer', 'small', '...",0
6318,"[we, have, an, immediate, opening, for, a, rec...","['immediate', 'opening', 'year', 'experience',...",1


## **Prepare Training/Testing Data**

In [40]:
# Prepare Train and Test Data sets
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(df['text_final'],df['fraudulent'],test_size=0.3)

## **Word Vectorization w TF-IDF**

In [41]:
# Word Vectorization using method TF-IDF
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text_final'])
X_train_Tfidf = Tfidf_vect.transform(X_train)
X_test_Tfidf = Tfidf_vect.transform(X_test)

# View the vocab its learned
print(Tfidf_vect.vocabulary_)
print(X_train_Tfidf)

{'customer': 1101, 'service': 4016, 'associate': 326, 'base': 444, 'indianapolis': 2223, 'right': 3827, 'candidate': 627, 'integral': 2312, 'part': 3182, 'talented': 4414, 'team': 4437, 'support': 4371, 'continued': 969, 'fulfill': 1872, 'office': 3019, 'request': 3756, 'come': 817, 'helpdesk': 2052, 'include': 2200, 'administrative': 88, 'paper': 3169, 'vend': 4786, 'machine': 2630, 'staff': 4237, 'conference': 909, 'room': 3855, 'set': 4026, 'ups': 4718, 'help': 2051, 'desk': 1233, 'center': 676, 'duty': 1391, 'responsibility': 3789, 'pull': 3545, 'appropriate': 259, 'furniture': 1890, 'inventory': 2361, 'meeting': 2741, 'require': 3758, 'table': 4404, 'chair': 692, 'complete': 870, 'return': 3806, 'dirty': 1293, 'place': 3311, 'bag': 427, 'area': 278, 'food': 1809, 'coordinator': 1003, 'coordinate': 1000, 'lunch': 2625, 'check': 709, 'supply': 4370, 'outlook': 3121, 'pop': 3347, 'situation': 4103, 'occur': 3013, 'need': 2918, 'address': 74, 'delivery': 1183, 'dock': 1338, 'applicabl

# **Train/ Evaluate Model**

In [42]:
# Classifier Algorithm= Support Vector Machine

# Fit the Classifier w the Training Data
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_Tfidf, Y_train)

# Use model to get predictions from the Testing data
predictions_SVM = SVM.predict(X_test_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score: ", accuracy_score(predictions_SVM, Y_test)*100)

SVM Accuracy Score:  93.24894514767934


## **Hypertune Model**

In [43]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 100],
              'gamma': [0.001, 0.01, .1, 1],
              'kernel': ['linear','rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVM, param_grid, verbose=3)

In [44]:
# Train the model with GridSearch
grid.fit(X_train_Tfidf, Y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] C=1, gamma=0.001, kernel=linear .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.924, total=   2.4s
[CV] C=1, gamma=0.001, kernel=linear .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV] ..... C=1, gamma=0.001, kernel=linear, score=0.930, total=   2.6s
[CV] C=1, gamma=0.001, kernel=linear .................................


...

[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.902, total=   2.5s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.902, total=   2.4s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.901, total=   2.3s
[CV] C=10, gamma=0.001, kernel=sigmoid ...............................
[CV] ... C=10, gamma=0.001, kernel=sigmoid, score=0.902, total=   2.2s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.914, total=   2.5s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.927, total=   2.5s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] ..... C=10, gamma=0.01, kernel=linear, score=0.922, total=   2.7s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV] .

[CV] .......... C=10, gamma=1, kernel=poly, score=0.926, total=   7.9s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.889, total=   1.9s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.903, total=   2.0s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.899, total=   1.9s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.915, total=   2.0s
[CV] C=10, gamma=1, kernel=sigmoid ...................................
[CV] ....... C=10, gamma=1, kernel=sigmoid, score=0.899, total=   1.9s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] ... C=100, gamma=0.001, kernel=linear, score=0.911, total=   2.4s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] .

[CV] ....... C=100, gamma=0.1, kernel=poly, score=0.902, total=   7.2s
[CV] C=100, gamma=0.1, kernel=poly ...................................
[CV] ....... C=100, gamma=0.1, kernel=poly, score=0.901, total=   7.3s
[CV] C=100, gamma=0.1, kernel=poly ...................................
[CV] ....... C=100, gamma=0.1, kernel=poly, score=0.902, total=   7.4s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.914, total=   2.4s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.927, total=   2.5s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.922, total=   2.5s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .... C=100, gamma=0.1, kernel=sigmoid, score=0.931, total=   2.5s
[CV] C=100, gamma=0.1, kernel=sigmoid ................................
[CV] .

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed: 15.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [45]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}
0.9378382800316999


In [46]:
# Train the hypertuned model
SVM2 = svm.SVC(kernel='rbf', C= 10, gamma= 1)
SVM2.fit(X_train_Tfidf, Y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [47]:
# Hypertuned Model Accuracy
print('SVM2 Acc Score: %.3f' % SVM2.score(X_test_Tfidf, Y_test))

SVM2 Acc Score: 0.942


In [48]:
from sklearn.metrics import classification_report
predictions = SVM2.predict(X_test_Tfidf)
print("SVM2 Classification Report: \n" + classification_report(Y_test, predictions, target_names=['real','fake']))

SVM2 Classification Report: 
              precision    recall  f1-score   support

        real       0.94      1.00      0.97      1700
        fake       0.97      0.45      0.62       196

    accuracy                           0.94      1896
   macro avg       0.95      0.73      0.79      1896
weighted avg       0.94      0.94      0.93      1896



## **Save Model**